# 🤖 Building AI Agents With Function Calling

In this notebook, you’ll learn how to turn a language model into an **agent**, a system that can not only talk, but also *act*.  
We do this by giving the model **tools**: small functions it can call to do real work like math, weather lookup, currency conversion, and more.

Function calling is how modern LLMs connect to the outside world. Instead of guessing answers, the model can decide: “Ah, this needs a tool,” and then request exactly the function and arguments it needs.

Your job in this notebook isn’t to write the tools themselves, those are provided. Your job is to:

- Define tool schemas  
- Inform the model which tools exist  
- Let the model choose when to use them  
- Run the tool it requests  
- Return the result so it can finish the response  

By the end, you’ll know how real AI agents are built and how to make LLMs interact with reliable code, APIs, and external systems.

Let’s get started. 🚀


In [1]:
from groq import Groq
import requests
import json
from functools import lru_cache
from sympy import symbols, sympify, simplify, solve, diff, integrate, expand, factor, latex

model_name = "llama-3.3-70b-versatile"

## What Are “Tools”?

**Tools** are capabilities you expose to the model. Each tool is defined using a standardized JSON structure. A tool can represent anything the model can call, such as:
- A Python function  
- A database query  
- An external API call  
- A local utility (file system, calculator, etc.)

The model decides *when* to call a tool and provides structured arguments.


### Tool Definition Format

Tools are defined as a list of objects, each describing a function the model is allowed to call:

```json
{
  "type": "function",
  "function": {
    "name": "function_name",
    "description": "What this function does.",
    "parameters": {
      "type": "object",
      "properties": {
        "...": { "type": "..." }
      },
      "required": [ ... ]
    }
  }
}
```

### Key Fields

| Field        | Purpose                                                     |
|--------------|-------------------------------------------------------------|
| `type`       | Must be `"function"` for function-calling tools             |
| `name`       | The name of your function or tool                           |
| `description`| Helps the model decide when the function is appropriate     |
| `parameters` | A JSON Schema describing valid arguments                    |
| `required`   | Ensures the model provides the needed fields                |


### Example

```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Retrieve the current weather for a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string"},
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    }
                },
                "required": ["city"]
            }
        }
    }
]
```

This tells the model:
- There is a function called **get_weather**
- The function retrieves the current weather for a given city
- It requires the city name as a string
- It also accepts an optional unit (`"celsius"` or `"fahrenheit"`)
- The model cannot call the function without providing a `city` argument

You can find more information and a complete guide regarding function calling at the following guide: [link](https://platform.openai.com/docs/guides/function-calling#page-top).

## 🧪 Building Our Own Tools!

In this section, we're going to create **several custom tools** but don’t worry, you won’t be writing the actual Python functions themselves. 🎉 **All the underlying functions will be provided for you.**  

Your job is to focus on the *AI side*: designing tool definitions, creating the schema, and teaching the model how to use what already exists.  


### 🔢 Warm-Up: Creating a Simple Addition Tool

Before we get into more complex examples, let's ease in with a warm-up exercise. We’ll start by creating a tool that simply **adds two numbers**.

In [2]:
def add_numbers(a: float, b: float) -> float:
    """
    Adds two numbers and returns the result.
    """
    return a + b

**`TODO:`** Define the `tools` object for the `add_numbers` function in the same fashion as above.

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Add two numbers.",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number"},
                    "b": {"type": "number"}
                },
                "required": ["a", "b"]
            }
        }
    }
]

## 🧠 Calling the Model With Our Tools

Now that we’ve defined our tools, it’s time to actually **use them**.  
In this step, we’ll send a request to the model *and* let it know which tools are available so it can decide whether one should be called.

This is where everything comes together: You pass the model a user message and the set of tools you defined, and the model determines whether one of those tools is appropriate for the task. 🚀

When you run the provided code, it will:

- Send a user request (for example, “Add 12 and 30”)  
- Tell the model about your available `tools`  
- Allow the model to autonomously decide whether to call one of them  
- Store the model’s response so you can inspect what it decided to do next

In the next step, we’ll take a look at the model’s response and see whether it chose to call your addition tool. 🧩

In [4]:
client = Groq()

resp = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": "Add 12 and 30"}],
    tools=tools
)

msg = resp.choices[0].message
display(msg)

ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=None, reasoning=None, tool_calls=[ChatCompletionMessageToolCall(id='y55441ce4', function=Function(arguments='{"a":12,"b":30}', name='add_numbers'), type='function')])

### 📝 Understanding the Model’s Response

When you inspect the model’s response, you’ll see something that looks like this:

```python
ChatCompletionMessage(content=None, role='assistant', annotations=None, executed_tools=None, function_call=None, reasoning=None, tool_calls=[ChatCompletionMessageToolCall(id='5r87sgs67', function=Function(arguments='{"a":12,"b":30}', name='add_numbers'), type='function')])
```


Let’s break down what this means.


Here’s what this means:

- The model saw the user request **"Add 12 and 30"** and decided that the best way to answer is to call a tool rather than generate normal text.
- `content=None` indicates there is **no natural-language response yet**, because the model is waiting for the tool to run.
- `role='assistant'` simply shows the message is coming from the assistant.
- The important part is the `tool_calls` list, which tells you that the model wants to execute a function-type tool.
- Inside the tool call, you can see:
  - the tool `type` is `"function"`
  - the tool `name` is `"add_numbers"` — the one you defined earlier
  - the `arguments` field contains `{"a":12,"b":30}`, meaning the model correctly extracted the numbers from the user request

Overall, this response shows that:

- the tool was recognized  
- the model chose it appropriately  
- it generated valid arguments in the correct schema  


**`TODO:`** To verify that everything is going well, come up with a prompt that **should not** cause the model to call any tool. This helps confirm that the model only uses tools when appropriate.

In [5]:
resp = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": "What's the capital of France?"}],
    tools=tools
)

msg_ = resp.choices[0].message
display(msg_)

ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', annotations=None, executed_tools=None, function_call=None, reasoning=None, tool_calls=None)

### 🧩 Executing the Tool and Completing the Interaction

After the model decides to call a tool, the general workflow is:

1. **The model requests a tool call** with the arguments it wants to use.
2. **You execute the tool yourself** (outside the model) using those arguments.
3. **You send the tool’s result back to the model** in a special “tool” message.
4. **The model uses that result** to generate its final, natural-language answer.

This two-step loop is how tool-enabled models work:  
the model identifies the tool it needs, you perform the action, and the model finishes the conversation using the result.


In [6]:
# If the model decided to call a tool
if msg.tool_calls:

    # We'll execute the tool and get the result
    tc = msg.tool_calls[0]
    args = json.loads(tc.function.arguments)
    result = add_numbers(**args)

    # Now, we send the model the tool's result so it can finalize its answer
    final = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "user", "content": "Add 12 and 30"},
            msg,
            {
                "role": "tool",
                "tool_call_id": tc.id,
                "content": str(result)
            }
        ]
    )

    print("Final answer:", final.choices[0].message.content)


Final answer: The answer is 42.


### 💸 Currency Conversion Tool

Now that you've successfully built and tested your first tool, it's time to move on to something a bit more practical. In this next section, we’ll create a tool that **converts currency**, for example, from USD to EUR, or GBP to JPY.

Just like before, the underlying conversion function `convert_currency` will be provided for you.  
Your job is to define the tool schema, describe its inputs, and let the model know how to use it.

To power this tool, we’ll be using the following public API:

**`https://api.frankfurter.app/latest?from=USD`**

#### 🧾 What is this API?

The Frankfurter API is a free, open-source currency exchange API that provides **latest foreign exchange rates**.  It’s great to use because:
- It **requires no API key**  
- It returns simple, easy-to-parse JSON  
- It supports many major currencies  

#### ⏳ A small caveat

The exchange rates are **updated relatively slowly** compared to financial-grade APIs. This is totally fine for learning purposes, but you should keep in mind that:
- The request rate limit is very low
- The rates may not reflect minute-by-minute market changes  
- It’s meant for demos, testing, and education, not live trading  

In [7]:
def get_rates():
    # Frankfurter API, no key required
    url = "https://api.frankfurter.app/latest?from=USD"
    resp = requests.get(url)

    if resp.status_code != 200:
        raise Exception("Failed to fetch currency rates")

    data = resp.json()
    return data["rates"]


def convert_currency(amount, from_currency, to_currency):
    rates = get_rates()

    from_currency = from_currency.upper()
    to_currency = to_currency.upper()

    if from_currency not in rates or to_currency not in rates:
        raise ValueError(f"Unsupported currency: {from_currency} or {to_currency}")

    # Convert to USD → then to target
    usd_value = amount / rates[from_currency]
    converted = usd_value * rates[to_currency]

    return round(converted, 2)

**`TODO:`** Using the function provided to you, define a proper **tool schema** for the currency converter.  
Your tool should clearly specify:

- the name of the tool  
- a short description of what it does  
- all required inputs (e.g., `from_currency`, `to_currency`, `amount`)  
- the correct JSON Schema types for each input  

After defining the tool, test it by sending the model a prompt that *should* trigger a currency conversion (for example: “Convert 50 USD to EUR”). Make sure the model selects your tool and generates valid arguments.

Once that happens, **execute the tool call yourself** using the provided function, and then **return its result back to the model** so it can produce the final natural-language answer.

*Hint:* The overall procedure is exactly the same as in the example above.

In [9]:
prompt = "Convert 150 EUR to JPY."

# Defining the tool schema for currency conversion
tools = [
    {
        "type": "function",
        "function": {
            "name": "convert_currency",
            "description": "Convert an amount of money from one currency to another using live exchange rates.",
            "parameters": {
                "type": "object",
                "properties": {
                    "amount": {"type": "number"},
                    "from_currency": {"type": "string"},
                    "to_currency": {"type": "string"}
                },
                "required": ["amount", "from_currency", "to_currency"]
            }
        }
    }
]

# Sending a request to the model to convert currency
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": prompt}
    ],
    tools=tools
)

message = response.choices[0].message

# If the model decided to call the currency conversion tool
if message.tool_calls:
    # We'll execute the tool and get the result
    t = message.tool_calls[0]
    args = json.loads(t.function.arguments)
    result = convert_currency(**args)

# Now, we send the model the tool's result so it can finalize its answer
followup = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": prompt},
        message,
        {
            "role": "tool",
            "tool_call_id": t.id,
            "content": str(result)
        }
    ]
)

print("Final answer:", followup.choices[0].message.content)

Final answer: The current exchange rate is 1 EUR = 180.898 JPY. 

150 EUR is approximately 27127 JPY.


### 🧮 A Symbolic Math Helper (SymPy)

Now we’re moving on to a more advanced and very powerful tool: a **general-purpose symbolic math assistant** built using **SymPy**.

This tool can:
- interpret a math expression written as a string  
- perform algebraic operations (simplification, factoring, expanding, solving, etc.)  
- perform calculus operations (derivatives, integrals, limits)  
- return the result in both **plain text** and **LaTeX**, so it’s suitable for readable output or rendering in notebooks

This tool is especially useful when you want the model to offload symbolic computation to a reliable math engine rather than attempt it purely through text generation.

In [10]:
def symbolic_math(task: str, expression: str, variable: str=None) -> dict:
    """
    Performs symbolic math operations using SymPy.
    Parameters:
    - task: The type of operation to perform (e.g., "simplify", "solve", "derivative", "integral", "expand", "factor").
    - expression: The mathematical expression as a string.
    - variable: The variable with respect to which to perform operations like derivative or integral (optional).
    Returns:
    - A dictionary with the result in string and LaTeX format, or an error message
    """
    try:
        # Parse expression safely
        expr = sympify(expression)

        # Optional variable parsing
        var = symbols(variable) if variable else None

        if task == "simplify":
            result = simplify(expr)

        elif task == "solve":
            result = solve(expr)

        elif task == "derivative":
            if var is None:
                raise ValueError("Derivative requires 'variable'")
            result = diff(expr, var)

        elif task == "integral":
            if var is None:
                raise ValueError("Integral requires 'variable'")
            result = integrate(expr, var)

        elif task == "expand":
            result = expand(expr)

        elif task == "factor":
            result = factor(expr)

        else:
            raise ValueError(f"Unknown task: {task}")

        # Return both raw string and LaTeX for LLM formatting
        return {
            "result_str": str(result),
            "result_latex": latex(result)
        }

    except Exception as e:
        return {"error": str(e)}

**`TODO:`** Using the SymPy-based function provided to you, define a **tool schema** for the symbolic math helper. Your schema should include:

- the tool’s name  
- a clear description of what the tool can do  
- all required inputs (e.g., `expression`, `task`)  
- appropriate JSON Schema types, such as strings or enums for the operation type  

After defining the tool, test it by sending the model prompts that *should* trigger symbolic math actions (for example: "Differentiate x^3 * sin(x) with respect to x."). Verify that the model selects your math tool and produces valid arguments.

Once the model requests the tool, **execute the SymPy function yourself**, then **return the result back to the model** so it can generate the final explanation or formatted answer.


In [11]:
prompt = "Differentiate x^3 * sin(x) with respect to x."

tools = [
    {
        "type": "function",
        "function": {
            "name": "symbolic_math",
            "description": "Perform symbolic math operations using SymPy: simplify, solve, derivative, integral, expand, or factor an expression.",
            "parameters": {
                "type": "object",
                "properties": {
                    "task": {
                        "type": "string",
                        "enum": ["simplify", "solve", "derivative", "integral", "expand", "factor"]
                    },
                    "expression": {"type": "string"},
                    "variable": {
                        "type": "string",
                        "description": "Variable with respect to which to differentiate or integrate",
                        "nullable": True
                    }
                },
                "required": ["task", "expression"]
            }
        }
    }
]

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": "Differentiate x^3 * sin(x) with respect to x."}
    ],
    tools=tools
)

message = response.choices[0].message

if message.tool_calls:
    call = message.tool_calls[0]
    args = json.loads(call.function.arguments)
    result = symbolic_math(**args)

followup = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": prompt},
        message,
        {
            "role": "tool",
            "tool_call_id": call.id,
            "content": json.dumps(result)
        }
    ]
)

print(followup.choices[0].message.content)

The derivative of x^3 * sin(x) with respect to x is x^3 * cos(x) + 3x^2 * sin(x).


## 🌦️ A Weather Retrieval System (Open-Meteo)

Our next tool is a more realistic, multi-step system that retrieves live weather information for a given city. This tool demonstrates how an agent can coordinate several external API calls to produce a polished result.

Here’s what the tool does:

- It takes a **city name** as input.
- It uses the **Open-Meteo Geocoding API** to convert that city into geographic coordinates (latitude and longitude).
- It then sends those coordinates to the **Open-Meteo Forecast API** to retrieve current weather details.
- Both API calls are **cached**, so repeated queries are faster and avoid redundant network usage.
- After getting the weather data, the tool:
  - converts the temperature into the user’s preferred unit (Celsius, Fahrenheit, or Kelvin),
  - collects wind speed, wind direction, and timestamp,
  - and returns a clean, structured summary containing the city, country, temperature, wind info, and time.
- If anything goes wrong (e.g., API failure, city not found), the tool returns a **helpful error message** instead.

In the end, the actual function you'll call for this tool is simply **`get_weather`**, which wraps all the steps above into one convenient interface.

This tool is a great example of how an LLM can collaborate with external systems to provide accurate, real-time information.

In [12]:
# Small cache to avoid repeated API calls
@lru_cache(maxsize=128)
def geocode(location: str):
    url = "https://geocoding-api.open-meteo.com/v1/search"
    resp = requests.get(url, params={"name": location, "count": 5, "language": "en"})
    
    if resp.status_code != 200:
        raise Exception("Geocoding service unavailable")

    data = resp.json()

    if "results" not in data or not data["results"]:
        raise ValueError(f"No matching city found for '{location}'")

    # Pick the first (best) result
    best = data["results"][0]

    return {
        "name": best["name"],
        "lat": best["latitude"],
        "lon": best["longitude"],
        "country": best.get("country", ""),
    }


@lru_cache(maxsize=128)
def fetch_weather(lat, lon):
    url = "https://api.open-meteo.com/v1/forecast"
    resp = requests.get(url, params={
        "latitude": lat,
        "longitude": lon,
        "current_weather": True
    })
    
    if resp.status_code != 200:
        raise Exception("Weather service unavailable")

    data = resp.json()

    if "current_weather" not in data:
        raise Exception("Weather data missing")

    return data["current_weather"]


def convert_temp(celsius, units):
    if units == "celsius":
        return celsius
    elif units == "fahrenheit":
        return celsius * 9/5 + 32
    elif units == "kelvin":
        return celsius + 273.15
    else:
        raise ValueError("Invalid units")


def get_weather(location: str, units: str = "celsius"):
    try:
        geo = geocode(location)
        weather = fetch_weather(geo["lat"], geo["lon"])

        temp_c = weather["temperature"]
        temp_u = convert_temp(temp_c, units)

        return {
            "location": geo["name"],
            "country": geo["country"],
            "temperature": round(temp_u, 2),
            "units": units,
            "wind_speed": weather["windspeed"],
            "wind_direction": weather["winddirection"],
            "time": weather["time"]
        }

    except Exception as e:
        return {"error": str(e)}

**`TODO:`** Using the provided `get_weather` function, create a **tool schema** that allows the model to request weather information. Your schema should include:
- the tool’s name (`get_weather`)  
- a clear description of what the tool does  
- the required inputs (e.g., `city`, `unit`)  
- correct JSON Schema types, and enums where appropriate (such as allowed temperature units)

After defining the tool, test it by prompting the model with a query that *should* trigger weather retrieval (e.g., "What is the weather in Tokyo right now in Fahrenheit?"). Check that the model selects your weather tool and produces valid arguments.

Finally, **execute the tool call using `get_weather`**, then **send its result back to the model** so it can generate the final, user-friendly weather report.


In [13]:
prompt = "What is the weather in Tokyo right now in Fahrenheit?"
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Retrieve current weather for a given city name",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City or place name to get weather for"
                    },
                    "units": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit", "kelvin"],
                        "default": "celsius"
                    }
                },
                "required": ["location"]
            }
        }
    }
]


response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": prompt}
    ],
    tools=tools
)

message = response.choices[0].message

if message.tool_calls:
    call = message.tool_calls[0]
    args = json.loads(call.function.arguments)
    result = get_weather(**args)

followup = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": prompt},
        message,
        {
            "role": "tool",
            "tool_call_id": call.id,
            "content": json.dumps(result)
        }
    ]
)

print(followup.choices[0].message.content)

The current weather in Tokyo is 44.42 degrees Fahrenheit.


### 📰 A News Retrieval and Article Extraction System

Our final tool in this sequence is a small but powerful news-fetching system. It demonstrates how an agent can interact with third-party APIs, process multiple results, and gather detailed information from the web.

Here’s what the tool does:

- It uses the **NewsAPI** service to search for recent articles based on a user-provided query.
- For each returned article, it attempts to download and extract the **full text** using the **Newspaper4k** library.
- The `extract_text` function handles downloading and parsing article content from a URL, and it is **cached** so that repeated requests for the same article do not trigger additional network calls.
- The `fetch_news` function:
  - performs the news search,
  - checks for errors (e.g., missing API key, invalid query),
  - processes each article by collecting its title, source, publication time, URL,
  - and includes either the extracted full text or, if extraction fails, the article’s description instead.

The final output is a clean, structured list of article details, or an informative error message if something goes wrong.

#### 🧩 Extra Exercise (Optional)

If you want to actually run this tool end-to-end, you’ll need to install the necessary libraries and register for an API key:

- Install the text extraction library (a modern fork of Newspaper3k):  
  **`pip install newspaper4k`**

- Register for a free NewsAPI key:  
  **https://newsapi.org/register**

This exercise is optional, but it’s a great way to experiment with live article retrieval and full-text extraction in a real agent pipeline.

In [ ]:
# Switch to True if you want to install newspaper4k
if False:
    !pip install newspaper4k
    from newspaper import Article
    NEWS_API_KEY = "YOUR_API_KEY"

In [15]:
# Caching to avoid excessive API calls
@lru_cache(maxsize=256)
def extract_text(url: str):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception:
        return ""


def fetch_news(query, max_articles=5, language="en"):
    try:
        # 1. Search for news articles
        url = "https://newsapi.org/v2/everything"
        params = {
            "q": query,
            "apiKey": NEWS_API_KEY,
            "sortBy": "publishedAt",
            "pageSize": max_articles,
            "language": language
        }

        resp = requests.get(url, params=params)
        data = resp.json()

        if resp.status_code != 200 or data.get("status") != "ok":
            return {"error": f"News API error: {data.get('message')}"}

        articles = data["articles"]

        cleaned = []

        # 2. Extract full text for each article
        for art in articles:
            url = art["url"]
            text = extract_text(url)

            cleaned.append({
                "title": art["title"],
                "source": art["source"]["name"],
                "url": url,
                "published_at": art["publishedAt"],
                "text": text if text else art.get("description", "")
            })

        return {
            "query": query,
            "results": cleaned
        }

    except Exception as e:
        return {"error": str(e)}

**`TODO:`** Using the provided `fetch_news` function, define a **tool schema** that allows the model to request news searches. Your schema should include:

- the tool’s name (`fetch_news`)  
- a clear description of what the tool does  
- the required inputs (e.g., `query`)  
- correct JSON Schema types 

After defining the tool, test it by prompting the model with a query that *should* trigger a news search (for example: "Give me a brief summary of the latest news about Ethereum."). Check that the model selects your news tool and produces valid arguments.

Once the model requests the tool, **execute the tool call using `fetch_news`**, then **return the result back to the model** so it can generate the final summary or explanation for the user.


In [16]:
prompt = "Give me a brief summary of the latest news about Ethereum."

tools = [
    {
        "type": "function",
        "function": {
            "name": "fetch_news",
            "description": "Fetch recent news articles about a topic and extract readable text.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Search terms for the news topic"
                    },
                    "max_articles": {
                        "type": "integer",
                        "default": 5
                    },
                    "language": {
                        "type": "string",
                        "default": "en"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": prompt}
    ],
    tools=tools
)

message = response.choices[0].message

if message.tool_calls:
    call = message.tool_calls[0]
    args = json.loads(call.function.arguments)
    result = fetch_news(**args)

followup = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": prompt},
        message,
        {
            "role": "tool",
            "tool_call_id": call.id,
            "content": json.dumps(result)
        }
    ]
)

print(followup.choices[0].message.content)

The latest news about Ethereum includes:

* Ethereum (ETH) is a cheaper per coin than Bitcoin (BTC), yet it keeps trailing in performance. Institutions favor Bitcoin's simple "digital gold" pitch and tight supply, while Ethereum's case is more complex.
* Bitcoin climbed back to $93,000 on Tuesday as El Salvador announced its largest single-day BTC purchase. Ethereum, XRP, and Dogecoin also rose 3%.
* Paxos Labs has launched USDG0, an omnichain extension of its regulated USDG stablecoin, bringing fully backed dollar liquidity to Hyperliquid, Plume, and Aptos through LayerZero's OFT standard.
* 0xbow, a Vitalik Buterin-backed privacy protocol, has closed a $3.5M round for compliant crypto privacy technology following Ethereum Foundation integration. The protocol has processed more than $6M in transaction volume.

These are just a few examples of the latest news about Ethereum. For more information, please visit the provided links to read the full articles.


## 🎯 Conclusion

You’ve now built several tools, connected them to a model, and experienced how function calling turns an LLM into an **agent** that can take actions instead of just generating text.  
This process — defining tools → letting the model choose → executing → returning results, is the fundamental loop behind modern AI agent systems.

Of course, this notebook is only an introduction. Agents can become far more capable when you add elements like planning, memory, multi-step reasoning, safety constraints, and orchestration frameworks. If you want to dive deeper, here are some recommended resources:

### 📚 Further Resources

**Building Effective Agents** — [link](https://www.anthropic.com/engineering/building-effective-agents)

A step-by-step guide that starts from “What is an agent?” and walks through how to design and use them effectively.

**Visual Guide to Agents & Tools** — [link](https://newsletter.maartengrootendorst.com/p/a-visual-guide-to-llm-agents)

A resource filled with helpful diagrams explaining how agents work, how they plan, and how tool calling fits into the overall loop.

**Hugging Face AI Agents Course** — [link](https://huggingface.co/learn/agents-course/en/unit0/introduction)

A more in-depth course. Useful as a general learning resource, specialized to the Hugging Face library.